In [1]:
features={}
features['Hamlet'] = 1
features['Container_ID'] = 2
features['Water_level'] = 3
features['Meso'] = 0
features['Container_position'] = 1
features['Treatment'] = 1



In [1]:
from sklearn.tree import DecisionTreeRegressor
treereg = DecisionTreeRegressor(max_depth=10, random_state=1)
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [2]:
def train():
    #read file
    from sklearn.model_selection import train_test_split
    import pandas as pd 
    import numpy as np
    data = pd.read_csv('final raw data.csv', na_filter=False)
    pos_data = data[data['Larvae'] >= 1]
    neg_data = data[data['Larvae'] == 0]
    
    # create final data set
    pos_len = len(pos_data.index)
    random_pos_data = pos_data.take(np.random.permutation(len(pos_data))[:pos_len])
    samples = int(0.75 * pos_len)
    train_pos_data = random_pos_data[:samples]
    test_pos_data = random_pos_data[samples:]
    final_pos_data = pd.concat([train_pos_data]*5)
    times = len(final_pos_data)
    final_neg_data = neg_data.take(np.random.permutation(len(neg_data))[:times])
    frames = [final_pos_data, final_neg_data]
    final_data = pd.concat(frames)
    
    # create X dataset 
    categories = ['Hamlet', 'Container_ID', 'Water_level', 'Container_position', 'Meso', 'Treatment']
    X = final_data[categories]
    
    #Classify Gaussian NB
    Y = final_data['Larvae']
    clf.fit(X, Y)
    GaussianNB(priors=None)

    #Tree Regression
    Y = final_data['Full count Larvae']
    treereg.fit(X, Y)
    
    return 

In [3]:
def predict(features):
    #predict if Larvae exists
    larvae = clf.predict(features)[0]
    print larvae
    if larvae == 1:
        numlarvae = treereg.predict(features)
        return True,numlarvae[0]
    else:
        return False,0



In [4]:
import sys
from flask import Flask, render_template, request
import pandas as pd
app = Flask(__name__)

#create a default route that supports both GET and POST methods
results = []
@app.route('/', methods=['GET', 'POST'])
def index():
    errors = []
#     results = []
    if request.method == "POST":
        # get url that the user has entered
        try:
            #print(request.form)
            features={}
            features['Hamlet'] = request.form['hamlet']
            features['Container_ID'] = request.form['containertype']
            features['Water_level'] = request.form['waterlevel']
            if 'meso' in request.form:
                features['Meso']=1
            else:
                features['Meso']=0
            if 'containerpos' in request.form:
                features['Container_position']=1
            else:
                features['Container_position']=0

            if 'chemicals' in request.form:
                features['Treatment']=1
            else:
                features['Treatment']=0
        except:
            print sys.exc_info()[0]
            errors.append(
                "Could not get data from HTML form. Please try again?"
            )
            return render_template('index.html', errors=errors)
        
        x = validateInput(features)
        if x!= 0:
            errors.append(x)
            
        #convert dict to dataframe for analysis
        y=[features]
        df=pd.DataFrame(y)
        #re-order to ensure dict conversion was good
        df = df[['Hamlet', 'Container_ID', 'Water_level', 'Container_position','Meso','Treatment']]
        
        ans = predict(df)
        
        hamlet=getHamlet(features['Hamlet'])
        container=getContainer(features['Container_ID'])
        if ans[0]==True:
            larvae="Yes"
        else:
            larvae="No"
        results.append([hamlet,container,larvae,str(ans[1])])
    return render_template('index.html', errors=errors, results=results)

def validateInput(features):
    try:
        features['Hamlet']=int(features['Hamlet'])
        features['Container_ID']=int(features['Container_ID'])
        features['Water_level']=int(features['Water_level'])
        features['Meso']=int(features['Meso'])
        features['Container_position']=int(features['Container_position'])
        features['Treatment']=int(features['Treatment'])
    except:
        return "Invalid input, please try again?"
    
    return 0

def getHamlet(hamletnum):
    hamlets = ["Tran An","Tan My","My Hanh","My Chanh","My Doi","Hoa Phu","Rach Doi","Ao Cho","Luang Khanh","Quang Hao","Tan Hanh"]
    if hamletnum >= 0 and hamletnum <=10:
        return hamlets[hamletnum]
    return ""

def getContainer(containernum):
    containers = ["Box Tank","Cylindrical Tank","Standard Jar","Small Jar","Large Jar","Other"]
    if containernum >= 1 and containernum <=6:
        return containers[containernum-1]
    return ""


In [ ]:
if __name__ == "__main__":
    print "Training model..."
    train()
    print "Completed Training..."
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Training model...


127.0.0.1 - - [14/Jan/2017 04:37:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2017 04:37:25] "POST / HTTP/1.1" 200 -


ImmutableMultiDict([('containertype', u'1'), ('containerpos', u'1'), ('hamlet', u'6'), ('waterlevel', u'10')])
0


127.0.0.1 - - [14/Jan/2017 04:37:43] "POST / HTTP/1.1" 200 -


ImmutableMultiDict([('containertype', u'3'), ('containerpos', u'1'), ('hamlet', u'0'), ('waterlevel', u'3')])
1
